In [10]:
import os
import globals
import sqlite3
from sqlite3 import Error

from pathlib import Path
from shutil import rmtree

In [12]:
sqlite_path = Path(globals.sqlite_term_vec_dir)
if sqlite_path.exists():
    rmtree(sqlite_path)
sqlite_path.mkdir(parents=True)

conn = sqlite3.connect(os.path.join(globals.sqlite_term_vec_dir,globals.sqlite_term_vec_file))
try:
    sql = "create table vectors (term nvarchar(512), avgbm25 float, vector text, primary key (term))"
    conn.execute(sql)
    sql = "create table tmp_vectors (term nvarchar(512), vector text, primary key (term))"
    conn.execute(sql)
except Error as e:
    print(e)


In [14]:
# Load the bm25 values into tmp table
count = 0
rows = []
c = conn.cursor()
with open(os.path.join(globals.bm25_dir, globals.bm25_file), encoding='utf-8') as fp: 
    while True: 
        line = fp.readline() 
        if not line: 
            break

        fields = line.split('\t')
        rows.append((fields[0], fields[1]))
        count += 1
        if count % 10000 == 0: 
            sql = 'insert into vectors (term, avgbm25) values (?,?)'
            c.executemany(sql, rows)
            conn.commit()
            print ("Inserted:", str(count))
            rows = []
if len(rows) > 0:
    sql = 'insert into vectors (term, avgbm25) values (?,?)'
    c.executemany(sql, rows)
    conn.commit()
    print ("Inserted:", str(count))



Inserted: 10000
Inserted: 20000
Inserted: 30000
Inserted: 40000
Inserted: 50000
Inserted: 60000
Inserted: 70000
Inserted: 80000
Inserted: 90000
Inserted: 100000
Inserted: 110000
Inserted: 120000
Inserted: 130000
Inserted: 137312


In [15]:
# load the vectors
count = 0
rows = []
c = conn.cursor()
with open(os.path.join(globals.vectors_out_dir, globals.vectors_out_file), encoding='utf-8') as fp: 
    while True: 
        line = fp.readline() 
        if not line: 
            break

        fields = line.split(' ')
        if len(fields) == 302:
            vec_str = ''
            for field in fields[1:301]:
                vec_str += field + '|'
            rows.append((fields[0], vec_str))
        count += 1
        if count % 10000 == 0: 
            sql = 'insert into tmp_vectors (term, vector) values (?,?)'
            c.executemany(sql, rows)
            conn.commit()
            print ("Inserted:", str(count))
            rows = []
if len(rows) > 0:
    sql = 'insert into tmp_vectors (term, vector) values (?,?)'
    c.executemany(sql, rows)
    conn.commit()
    print ("Inserted:", str(count))


Inserted: 10000
Inserted: 19250


In [17]:
sql = 'update vectors set vector = (select vector from tmp_vectors where vectors.term = tmp_vectors.term)'
c.execute(sql)
conn.commit()

sql = 'drop table tmp_vectors'
c.execute(sql)
conn.commit()

In [20]:
# shrink db
sql = 'vacuum'
c.execute(sql)
conn.commit()